In [95]:
from enum import Enum
from random import shuffle, randint
import pandas as pd
import random

class Card(Enum):
    PIKA_EX = 1
    BASIC = 2
    P_BALL = 3
    P_RE = 4
    X_SP = 5
    OTHER = 6

class BoardState:
    active_slot = []
    bench = []
    energy = False
    basic_energy_count = 0
    pika_energy_count = 0
    damage_counter = 0

    def reset():
        BoardState.active_slot = []
        BoardState.bench = []
        BoardState.energy = False
        BoardState.basic_energy_count = 0
        BoardState.pika_energy_count = 0
        BoardState.damage_counter = 0

class Simulation: 
    
    #Game start: initiate hand and deck shuffled
    def __init__(self, turn, basic_count):
        self.basic_count = basic_count
        self.turn = turn
        self.hand = []
        self.deck = [Card.PIKA_EX] * 2 + \
                    [Card.P_BALL] * 2 + \
                    [Card.P_RE] * 2 + \
                    [Card.X_SP] * 2 + \
                    [Card.BASIC] * basic_count + \
                    [Card.OTHER] * (12 - basic_count)
        shuffle(self.deck)

    def display_bs(self):
        print(BoardState.active_slot)
        print(BoardState.bench)

    #Remove a card from deck and place it in your hand
    def draw_card(self, times=1):
        for card in range(times):
            try:
                self.hand.append(self.deck.pop())
            except:
                #print('Nothing in deck to draw!')
                pass

    def turn_0(self):
        
        #Put PIKA_EX or BASIC into the active slot if empty
        if not BoardState.active_slot:
            if Card.PIKA_EX in self.hand:
                BoardState.active_slot = Card.PIKA_EX
                self.hand.remove(Card.PIKA_EX)
            else:
                BoardState.active_slot = Card.BASIC
                self.hand.remove(Card.BASIC)

        #Fill the bench with PIKA_EX/BASIC up to 3
        while len(BoardState.bench) < 3:
            if Card.PIKA_EX in self.hand:
                BoardState.bench.append(Card.PIKA_EX)
                self.hand.remove(Card.PIKA_EX)
            elif Card.BASIC in self.hand and (BoardState.bench + [BoardState.active_slot]).count(Card.BASIC) < 3:   #Won't fill the board with all basics (leaves room for PIKA_EX)
                BoardState.bench.append(Card.BASIC)
                self.hand.remove(Card.BASIC)
            else:
                break

        return len(BoardState.bench + [BoardState.active_slot])

        #self.display_bs()

    def player_turn(self):
        
        self.draw_card(1)

        #Energy generation; 1 energy available per turn, with the exception of player turn 1
        if self.turn != 0:
            BoardState.energy = True

        #Play available card draw: first P_RE then P_BALL to maximize chances of pulling PIKA_EX/filling the board
        try:
            if Card.P_RE in self.hand:
                #print('Playing Professors Research')
                self.hand.remove(Card.P_RE)
                self.draw_card(2)
            while Card.P_BALL in self.hand:
                #print('Playing Pokeball')
                self.hand.remove(Card.P_BALL)
                index_choice = random.choice([i for i, x in enumerate(self.deck) if x == Card.BASIC or x == Card.PIKA_EX])
                self.hand.append(self.deck[index_choice])
        except:
            #print('Nothing in deck to draw!')
            pass

        #Play newly drawn PIKA_EX or BASIC cards
        while len(BoardState.bench) < 3:
            if Card.PIKA_EX in self.hand:
                BoardState.bench.append(Card.PIKA_EX)
                self.hand.remove(Card.PIKA_EX)
            elif Card.BASIC in self.hand and (BoardState.bench + [BoardState.active_slot]).count(Card.BASIC) < 3:   #Won't fill the board with all basics (leaves room for PIKA_EX)
                BoardState.bench.append(Card.BASIC)
                self.hand.remove(Card.BASIC)
            else:
                break

        #Engage PIKA_EX into active slot if not already, by means of X_SP or existing energy count on a BASIC
        if BoardState.active_slot != Card.PIKA_EX and Card.PIKA_EX in BoardState.bench:
            if Card.X_SP in self.hand:
                #print('Playing X-Speed')
                self.hand.remove(Card.X_SP)
                BoardState.active_slot = Card.PIKA_EX
                BoardState.bench.remove(Card.PIKA_EX)
                BoardState.bench.append(Card.BASIC)
            elif BoardState.basic_energy_count > 0:
                BoardState.basic_energy_count = 0
                BoardState.active_slot = Card.PIKA_EX
                BoardState.bench.remove(Card.PIKA_EX)
                BoardState.bench.append(Card.BASIC)
        
        #Place energy on whatever is in the active slot, UNLESS it is a BASIC and already has one
        while BoardState.energy == True:
            if Card.PIKA_EX == BoardState.active_slot:
                BoardState.pika_energy_count += 1
            elif Card.BASIC == BoardState.active_slot:
                BoardState.basic_energy_count += 1
            BoardState.energy = False

        #Attack when PIKA_EX reaches 2 energy
        if BoardState.pika_energy_count >= 2:
            BoardState.damage_counter += len(BoardState.bench) * 30

        #self.display_bs()
        #print('Pika energy count: ', BoardState.pika_energy_count)
        #print('Basic energy count ', BoardState.basic_energy_count)
        #print('Damage: ', BoardState.damage_counter)

        #End turn
        if self.turn == 0:
            self.turn += 2
            return 1, len(BoardState.bench + [BoardState.active_slot]), BoardState.pika_energy_count, BoardState.damage_counter
        else:
            self.turn += 1
            return self.turn-1, len(BoardState.bench + [BoardState.active_slot]), BoardState.pika_energy_count, BoardState.damage_counter

    

    def runSimulation(self):

        #Create dataframe
        df = pd.DataFrame(columns=['turn','board_mons','pika_energy','damage_counter'])

        #Draw your starting hand, if PIKA_EX or BASIC isn't drawn, redraw your hand
        while not (Card.PIKA_EX in self.hand or Card.BASIC in self.hand):
            self.hand = []
            shuffle(self.deck)
            self.draw_card(5)

        #Conduct first player turn, fill dataframe with turn 0 info
        df.loc[0] =[0, self.turn_0(), 0, 0]

        #Play through 7 turns
        for turn in range(1,8):
            df.loc[turn] = self.player_turn()
        
        return df

In [105]:
#Starting tails (1), basic count of 4 (4)
BoardState.reset()
print(Simulation(1, 4).runSimulation())

   turn  board_mons  pika_energy  damage_counter
0     0           3            0               0
1     1           4            1               0
2     2           4            2              90
3     3           4            3             180
4     4           4            4             270
5     5           4            5             360
6     6           4            6             450
7     7           4            7             540
